# Evaluating Strands Agent with Observability with LangFuse

## Overview
In this module, we'll explore how to implement observability for [Strands Agent](https://strandsagents.com/latest/) using [Langfuse](https://langfuse.com/). Strands Agent enables you to build an intelligent AWS assistant by connecting foundation models to AWS services and resources, here's the link to [Strands Agent SDK's GitHub repo](https://github.com/strands-agents). While powerful AI agents can solve complex problems, understanding their behavior and performance requires robust observability solutions.

Langfuse integration provides comprehensive visibility into your agent's operations, helping you monitor, debug, and optimize its performance. This module will guide you through implementing and leveraging Langfuse observability with Strands Agent to create more reliable, transparent, and effective AI solutions.

Here's what you will learn from this module:
1. Setup Lanfuse Observability 
2. Create a Strands Agent with [built-in tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/example-tools-package/) and trace it with Lanfuse
3. Create a Strands Agent with [custom tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/python-tools/) and trace it with Lanfuse
4. Lastly create a Strands Agent with [MCP tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/mcp-tools/) and trace it with Lanfuse

## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                              |
|--------------------|---------------------------------------------------------|
|Native tools used   |image_reader                                             |
|Custom tools created|aws_health_status_checker                                |
|MCP Tools           |aws_documentation_mcp_tool, aws_pricing_mcp_tool         |
|Agent Structure     |Single agent architecture                                |
|AWS services used   |Amazon Bedrock                                           |
|Integrations        |LangFuse for observability                               |

</div>



## Architecture

<div style="text-align:left">
    <img src="./image/architecture.png" width="75%" />
</div>

## Key Features
- Fetches Strands agent interaction traces from Langfuse. You can also save these traces offline and use them here without Langfuse.
- Evaluates conversations using specialized metrics for agents, tools
- Pushes evaluation scores back to Langfuse for a complete feedback loop
- Evaluate both single-turn (with context) and multi-turn conversations

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.5 on Amazon Bedrock
* LangFuse Key

Let's now install the requirement packages for our Strands Agent

In [6]:
# Uncomment the following line to install dependencies if you are not using AWS workshop environment
%pip install -q langfuse==2.59.7  boto3  --upgrade

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Install required packages
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
!uvx awslabs.aws-pricing-mcp-server@latest

Installed 49 packages in 137ms                                       
^C
  + Exception Group Traceback (most recent call last):
  |   File "/home/sagemaker-user/.cache/uv/archive-v0/U4CfRWVbDZ4mKxyLcRo_D/bin/awslabs.aws-pricing-mcp-server", line 12, in <module>
  |     sys.exit(main())
  |   File "/home/sagemaker-user/.cache/uv/archive-v0/U4CfRWVbDZ4mKxyLcRo_D/lib/python3.10/site-packages/awslabs/aws_pricing_mcp_server/server.py", line 1356, in main
  |     mcp.run()
  |   File "/home/sagemaker-user/.cache/uv/archive-v0/U4CfRWVbDZ4mKxyLcRo_D/lib/python3.10/site-packages/mcp/server/fastmcp/server.py", line 250, in run
  |     anyio.run(self.run_stdio_async)
  |   File "/home/sagemaker-user/.cache/uv/archive-v0/U4CfRWVbDZ4mKxyLcRo_D/lib/python3.10/site-packages/anyio/_core/_eventloop.py", line 74, in run
  |     return async_backend.run(func, args, {}, backend_options)
  |   File "/home/sagemaker-user/.cache/uv/archive-v0/U4CfRWVbDZ4mKxyLcRo_D/lib/python3.10/site-packages/anyio/_backends

In [3]:
!echo '{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {}, "clientInfo": {"name": "test", "version": "1.0.0"}}}' | uvx awslabs.aws-pricing-mcp-server@latest

Installed 49 packages in 79ms                                        
{"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":false}},"serverInfo":{"name":"awslabs.aws-pricing-mcp-server","version":"1.13.0"},"instructions":"This server provides two primary functionalities:\n\n    # USE CASE 1: AWS SERVICE CATALOG & PRICING DISCOVERY\n    Access AWS service catalog information and pricing details through a structured workflow:\n\n    1. Discovery Workflow:\n       - get_pricing_service_codes: Retrieve all available AWS service codes (starting point)\n       - get_pricing_service_attributes: Get filterable attributes for a specific service\n       - get_pricing_attribute_values: Get possible values for a specific attribute\n       - get_pricing: Get actual pricing data with optional filters\n       - get_price_list_urls: Get bulk pricing da

In [3]:
# if you already define the environment variables in the .env of the vscode server, please skip the following cell
# Define the environment variables for langfuse
# You can find those values when you create the API key in Langfuse
# import os
# os.environ["LANGFUSE_SECRET_KEY"] = "xxxx" # Your Langfuse project secret key
# os.environ["LANGFUSE_PUBLIC_KEY"] = "xxxx" # Your Langfuse project public key
# os.environ["LANGFUSE_HOST"] = "xxx" # Langfuse domain

### Importing dependency packages

Now let's import the dependency packages

In [1]:
import os
import json
import base64
from typing import Dict, Any, List, Optional
from datetime import datetime
import uv

from strands import Agent, tool
from strands_tools import calculator, image_reader
import sys
sys.path.append(os.path.abspath('..'))  # Add parent directory to path

from utils import *

# Import MCP related modules
from mcp import StdioServerParameters, stdio_client
from strands.tools.mcp import MCPClient

# Import Strands telemetry for OpenTelemetry integration
from strands.telemetry import StrandsTelemetry

In [10]:
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

## Setting Strands Agents to emit LangFuse traces
#### The first step here is to set Strands Agents to emit traces to LangFuse

#### Configure the telemetry(Creates new tracer provider and sets it as global)

In [2]:
secret=get_secret()
secrets=json.loads(secret)

In [3]:
# Get keys for your project from the project settings page: https://cloud.langfuse.com
langfuse_public_key = secrets["LANGFUSE_PUBLIC_KEY"]
langfuse_secret_key = secrets["LANGFUSE_SECRET_KEY"]
langfuse_host = secrets['LANGFUSE_HOST'] #, 'https://cloud.langfuse.com')

if langfuse_public_key and langfuse_secret_key:
    LANGFUSE_AUTH = base64.b64encode(
        f"{langfuse_public_key}:{langfuse_secret_key}".encode()
    ).decode()
    
    os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = langfuse_host + "/api/public/otel"
    os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"
    print("Langfuse observability configured.\n")

else:
    print("Warning: Langfuse API keys not found. Observability will not be enabled.")

Langfuse observability configured.



In [4]:
try:
    strands_telemetry = StrandsTelemetry().setup_otlp_exporter()
    print("OpenTelemetry exporter configured for Langfuse.")
except ImportError:
    print("Warning: OpenTelemetry exporter not available. Install with 'pip install opentelemetry-exporter-otlp'.")
    strands_telemetry = None

OpenTelemetry exporter configured for Langfuse.


## Build your first strands agent
### Using the built in tool
Tools are the primary mechanism for extending agent capabilities, enabling them to perform actions beyond simple text generation. Tools allow agents to interact with external systems, access data, and manipulate their environment.
Strands offers built-in example tools to get started quickly experimenting with agents and tools during development. For more information, see Example [Built-in Tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/example-tools-package/).

In [5]:
# Customer query
query = """
What is in the picture of image/aws-architecture.png in the current folder path
"""

# Building the strands agent with built-in tool image_read and including Lanfuse for tracing
agent = Agent(
    tools=[image_reader],
    model="us.anthropic.claude-3-5-sonnet-20241022-v2:0", # Using Claude 3.5 Sonnet via Bedrock
    name="strands-agent-built-in-tool-example",
    trace_attributes={
        "session.id": "aws-mcp-agent-demo-session",
        "user.id": "xx-yy@example.com",
        "langfuse.tags": [
            "AWS-Strands-Agent",
            "Built-In-Tool",
        ],
        "metadata": {
            "environment": "development",
            "version": "1.0.0",
            "query_type": "storage_recommendation"
        }
    }
)
    
try:
    # Run the agent and get the response within the MCP client context
    response = agent(query)
except Exception as e:
    print(f"\nError: {e}")

Let me help you retrieve and examine that image.
Tool #1: image_reader
This image shows an AWS (Amazon Web Services) microservice architecture diagram. It's divided into three main sections:

1. User Interface:
- Contains Amazon CloudFront (content delivery network)
- Amazon S3 (storage service)

2. Compute Implementation:
- ALB (Application Load Balancer)
- Amazon ECS (Elastic Container Service)

3. Data Store:
- Amazon ElastiCache (caching service)
- Amazon Aurora (relational database)
- Amazon DynamoDB (NoSQL database)

The architecture flow shows how these services are connected:
- CloudFront connects to S3 for static content
- Traffic flows through the ALB to ECS
- ECS then connects to various data stores (ElastiCache, Aurora, and DynamoDB)

This appears to be a typical microservice architecture pattern in AWS, designed for scalability and separation of concerns between different components of the application.

### Now go to your Lanfuse console, you should be able to see a new trace with the name "strands_agent_built-in-tool-example" appear. Have a look at what's in the trace.

<div style="text-align:left">
    <img src="./image/trace1.jpg" width="15%" />
    <img src="./image/trace2.png" width="75%" />
</div>

## Strands Agents with custom tools
Define tools by creating Python modules that contain a tool specification and a matching function. This approach gives you more control over the tool's definition and is useful for dependency-free implementations of tools.

In [6]:
from typing import Dict, List, Optional
import requests
import xml.etree.ElementTree as ET

# Using the @tool python decorator, creating a custom tool that uses the AWS Health Dashboard RSS feed URL to check AWS service status
@tool
def aws_health_status_checker(region: str = "us-east-1", service_name: Optional[str] = None) -> Dict:
    """
    Check the current operational status of AWS services in a specific region using the public RSS feed.

    Args:
        region: AWS region to check (e.g., us-east-1, us-west-2)
        service_name: Optional specific service to check (e.g., ec2, s3, lambda)
                     If not provided, will return status for all services

    Returns:
        Dictionary containing service health information
    """
    try:
        # AWS Health Dashboard RSS feed URL
        rss_url = "https://status.aws.amazon.com/rss/all.rss"

        # Get the RSS feed
        response = requests.get(rss_url, timeout=10)
        response.raise_for_status()

        # Parse the XML
        root = ET.fromstring(response.content)

        # Find all items (service events)
        items = root.findall(".//item")

        # Filter events by region and service
        events = []
        for item in items:
            title = item.find("title").text if item.find("title") is not None else ""
            description = item.find("description").text if item.find("description") is not None else ""
            pub_date = item.find("pubDate").text if item.find("pubDate") is not None else ""
            link = item.find("link").text if item.find("link") is not None else ""

            # Check if this event is for the requested region
            if region.lower() in title.lower():
                # If service_name is specified, check if this event is for that service
                if service_name is None or service_name.lower() in title.lower():
                    events.append({
                        "title": title,
                        "description": description,
                        "published_date": pub_date,
                        "link": link
                    })

        if not events:
            return {
                "status": "healthy",
                "message": f"All services in {region} appear to be operating normally" if not service_name else f"{service_name} in {region} appears to be operating normally",
                "events": []
            }

        return {
            "status": "service_disruption",
            "message": f"Service disruptions detected in {region}",
            "events": events
        }

    except Exception as e:
        return {
            "status": "error",
            "message": f"Error checking AWS service status: {str(e)}",
            "events": []
        }

#### Create a Strands Agent with this custom tool

In [7]:
# Customer query
query = """
Check AWS service status in the us-east-1 region. focus on ec2 instances 
"""

# Building the strands agent with the custom tool "aws_health_status_checker" defined above 
agent = Agent(
    system_prompt="You are an AWS service checking agent, use the aws_service_status_checker to tell the user if the service status that they ask",
    tools=[aws_health_status_checker],
    model="us.anthropic.claude-3-5-sonnet-20241022-v2:0", # Using Claude 3.5 Sonnet via Bedrock
    name="strands-agent-custom-tool-example",
    trace_attributes={
        "session.id": "aws-mcp-agent-demo-session",
        "user.id": "example-user@example.com",
        "langfuse.tags": [
            "AWS-Strands-Agent",
            "Custom-Tool",
        ],
        "metadata": {
            "environment": "development",
            "version": "1.0.0",
            "query_type": "storage_recommendation"
        }
    }
)
    
try:
    # Run the agent and get the response within the MCP client context
    response = agent(query)
except Exception as e:
    print(f"\nError: {e}")

I'll help you check the status of EC2 service in the us-east-1 region using the AWS health status checker.
Tool #1: aws_health_status_checker
Based on the results, EC2 service in the us-east-1 region is currently healthy and operating normally. There are no reported issues or service disruptions at this time. You can proceed with your EC2-related operations in this region without any known service constraints.

Is there anything specific about EC2 or any other AWS service you'd like me to check for you?

### Now go to your Lanfuse console again, you should be able to see a new trace with the name "strands-agent-custom-tool-example" appear. Have a look at what's in the trace.

## Strands Agents with MCP servers
The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands Agents integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. Strands includes built-in support for connecting to MCP servers and using their tools.

When working with MCP tools in Strands, all agent operations must be performed within the MCP client's context manager (using a with statement). This requirement ensures that the MCP session remains active and connected while the agent is using the tools. If you attempt to use an agent or its MCP tools outside of this context, you'll encounter errors because the MCP session will have closed.

#### Initialize MCP client for AWS documentation and AWS Pricing

In [8]:
import subprocess
import shutil

# Check if uvx is available
uvx_path = shutil.which("uvx")
print(f"uvx command found at: {uvx_path}")

if not uvx_path:
    print("❌ uvx command not found")
    # Try alternative commands
    for alt_cmd in ["uv", "pip", "pipx"]:
        if shutil.which(alt_cmd):
            print(f"✅ Alternative found: {alt_cmd}")

uvx command found at: /opt/conda/bin/uvx


from strands.tools.mcp import MCPClient
from mcp.client.stdio import StdioServerParameters, stdio_client
import sys

# Use direct Python module execution
aws_docs_mcp = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command=sys.executable,
            args=["-m", "aws_documentation_mcp_server"]  # Adjust module name
        )
    )
)
aws_pricing_mcp = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command=sys.executable,
            args=["-m", "aws_pricing_mcp_server"]  # Adjust module name
        )
    )
)

In [5]:
aws_docs_mcp = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", 
            args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Initialize MCP client for AWS pricing
aws_pricing_mcp = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", 
            args=["awslabs.aws-pricing-mcp-server@latest"]
        )
    )
)

In [6]:
print("AWS Pricing MCP Configuration:")
print(f"Command: {aws_pricing_mcp._command if hasattr(aws_pricing_mcp, '_command') else 'Unknown'}")
print(f"Args: {aws_pricing_mcp._args if hasattr(aws_pricing_mcp, '_args') else 'Unknown'}")
print(f"Env: {aws_pricing_mcp._env if hasattr(aws_pricing_mcp, '_env') else 'Unknown'}")


AWS Pricing MCP Configuration:
Command: Unknown
Args: Unknown
Env: Unknown


#### Test query that requires both AWS documentation and cost estimation

In [7]:
mcp_tools = []

# Initialize AWS Docs MCP
try:
    with aws_docs_mcp:
        docs_tools = aws_docs_mcp.list_tools_sync()
        mcp_tools.extend(docs_tools)
        print(f"AWS Docs MCP: {len(docs_tools)} tools loaded")
except Exception as e:
    print(f"AWS Docs MCP failed: {e}")

AWS Docs MCP: 3 tools loaded


In [ ]:
query = """
I'm planning to use S3 for storing 1TB of data. 
1. What storage class would you recommend for data that is accessed infrequently based on the aws document?
2. Can you estimate the monthly cost for storing 1TB in this storage class?
"""

print(f"\n=== QUERY: {query} ===\n")
print("Processing query...")

# IMPORTANT: Use the MCP client within a context manager
with aws_docs_mcp, aws_pricing_mcp:
    # Get the available tools from the MCP server
    mcp_tools = aws_docs_mcp.list_tools_sync() + aws_pricing_mcp.list_tools_sync()
    print(f"Available MCP tools: {len(mcp_tools)} tools found")
    
    # Create the agent with our custom tools and MCP tools
    agent = Agent(
        system_prompt="""You are an AWS assistant specialized in helping users with AWS-related tasks.
        You can provide information about AWS services, estimate costs, and help manage AWS resources.
        
        When answering questions:
        1. Use the AWS documentation when needed to provide accurate information
        2. Provide clear explanations with examples
        3. Consider best practices and security recommendations
        4. Be precise and accurate in your responses
        
        For resource tagging, use the tag_aws_resources tool with the resource ID and tag key-value pairs.
        For AWS documentation queries, use the AWS documentation MCP tools.
        For cost estimation, use the AWS Pricing MCP tools.
        """,
        tools=[
            # MCP tools
            *mcp_tools
        ],
        model="us.anthropic.claude-3-5-sonnet-20241022-v2:0", # Using Claude 3.5 Sonnet via Bedrock
        name="strands-agent-mcp_example",
        trace_attributes={
            "session.id": "aws-mcp-agent-demo-session",
            "user.id": "example-user@example.com",
            "langfuse.tags": [
                "AWS-MCP-Agent",
                "MCP"
            ],
            "metadata": {
                "environment": "development",
                "version": "1.0.0",
                "query_type": "storage_recommendation"
            }
        }
    )
    
    try:
        # Run the agent and get the response within the MCP client context
        response = agent(query)
        #print("\nAgent messages:")
        #print(json.dumps(agent.messages, indent=4))
    except Exception as e:
        print(f"\nError: {e}")

DEBUG | strands.tools.mcp.mcp_client | [Thread: MainThread, Session: eaadab4c-8aec-411b-b7ba-db0a9a99aca7] entering MCPClient context



=== QUERY: 
I'm planning to use S3 for storing 1TB of data. 
1. What storage class would you recommend for data that is accessed infrequently based on the aws document?
2. Can you estimate the monthly cost for storing 1TB in this storage class?
 ===

Processing query...


#### Now you know the drill, go to your Lanfuse console, you should be able to see a new trace with the name "strands-agent-mc-example" appear. Have a look at what's in the trace.

#### You can also use the built in tracing to show what the agent traces locally

In [9]:
print("\nAgent messages:")
print(json.dumps(agent.messages, indent=4))


Agent messages:
[
    {
        "role": "user",
        "content": [
            {
                "text": "\nI'm planning to use S3 for storing 1TB of data. \n1. What storage class would you recommend for data that is accessed infrequently based on the aws document?\n2. Can you estimate the monthly cost for storing 1TB in this storage class?\n"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "I'll help you with both questions.\n\n1. First, let me search AWS documentation about S3 storage classes for infrequently accessed data."
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_rFQW4720ThakwRgN1EAfbQ",
                    "name": "search_documentation",
                    "input": {
                        "search_phrase": "S3 storage classes infrequent access"
                    }
                }
            }
        ]
    },
    {
        "role": "user"

You can go to the Langfuse console to see the details of the traces.

## Lab4 Summary:

In Lab 4, we explored advanced observability for **Strands Agents** using `Langfuse`, demonstrating three distinct
tool integration approaches with comprehensive tracing capabilities. 

We successfully implemented built-in tools using the `image_reader` for AWS architecture analysis, created custom tools like the 
`aws_health_status_checker` for real-time service monitoring, and integrated `MCP (Model Context Protocol)` tools combining AWS Documentation and Pricing servers for complex multi-tool workflows. The key achievement was establishing complete observability through OpenTelemetry integration with StrandsTelemetry, enabling real-time monitoring of agent decision-making processes, tool usage patterns, and performance metrics in the Langfuse console. 

As we conclude this lab, you've gained valuable experience in building transparent, monitorable AI agents that transform from black boxes into production-ready systems with comprehensive observability, preparing you for deploying robust agentic systems with confidence in enterprise environments.

# Evaluating Strands Agent with Observability with LangFuse

## Overview
In this module, we'll explore how to implement observability for [Strands Agent](https://strandsagents.com/latest/) using [Langfuse](https://langfuse.com/). Strands Agent enables you to build an intelligent AWS assistant by connecting foundation models to AWS services and resources, here's the link to [Strands Agent SDK's GitHub repo](https://github.com/strands-agents). While powerful AI agents can solve complex problems, understanding their behavior and performance requires robust observability solutions.

Langfuse integration provides comprehensive visibility into your agent's operations, helping you monitor, debug, and optimize its performance. This module will guide you through implementing and leveraging Langfuse observability with Strands Agent to create more reliable, transparent, and effective AI solutions.

Here's what you will learn from this module:
1. Setup Lanfuse Observability 
2. Create a Strands Agent with [built-in tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/example-tools-package/) and trace it with Lanfuse
3. Create a Strands Agent with [custom tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/python-tools/) and trace it with Lanfuse
4. Lastly create a Strands Agent with [MCP tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/mcp-tools/) and trace it with Lanfuse

## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                              |
|--------------------|---------------------------------------------------------|
|Native tools used   |image_reader                                             |
|Custom tools created|aws_health_status_checker                                |
|MCP Tools           |aws_documentation_mcp_tool, aws_pricing_mcp_tool         |
|Agent Structure     |Single agent architecture                                |
|AWS services used   |Amazon Bedrock                                           |
|Integrations        |LangFuse for observability                               |

</div>



## Architecture

<div style="text-align:left">
    <img src="./image/architecture.png" width="75%" />
</div>

## Key Features
- Fetches Strands agent interaction traces from Langfuse. You can also save these traces offline and use them here without Langfuse.
- Evaluates conversations using specialized metrics for agents, tools
- Pushes evaluation scores back to Langfuse for a complete feedback loop
- Evaluate both single-turn (with context) and multi-turn conversations

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.5 on Amazon Bedrock
* LangFuse Key

Let's now install the requirement packages for our Strands Agent

In [2]:
# Uncomment the following line to install dependencies if you are not using AWS workshop environment
# %pip install -q langfuse boto3  --upgrade

In [ ]:
# Install required packages
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 9.9 MB/s eta 0:00:00
  Using cached opentelemetry_exporter_otlp-1.36.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached docstring_parser-0.17.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached opentelemetry_api-1.36.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_instrumentation_threading-0.57b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_sdk-1.36.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached watchdog-6.0.0-py3-none-manylinux2014_x86_64.whl.metadata (44 kB)
  Using cached aws_requests_auth-0.4.3-py2.py3-none-any.whl.metadata (567 bytes)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached pillow-11.3.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (9.0 kB)
  Using cached prompt_toolkit-3.0.51-py3-none-any.whl.metadata (6.4 kB)
  Using cached PyJWT-2.10.1-py3-n

In [3]:
# if you already define the environment variables in the .env of the vscode server, please skip the following cell
# Define the environment variables for langfuse
# You can find those values when you create the API key in Langfuse
# import os
# os.environ["LANGFUSE_SECRET_KEY"] = "xxxx" # Your Langfuse project secret key
# os.environ["LANGFUSE_PUBLIC_KEY"] = "xxxx" # Your Langfuse project public key
# os.environ["LANGFUSE_HOST"] = "xxx" # Langfuse domain

### Importing dependency packages

Now let's import the dependency packages

In [4]:
import os
import json
import base64
from typing import Dict, Any, List, Optional
from datetime import datetime
import uv

from strands import Agent, tool
from strands_tools import calculator, image_reader
import sys
sys.path.append(os.path.abspath('..'))  # Add parent directory to path

from utils import *

# Import MCP related modules
from mcp import StdioServerParameters, stdio_client
from strands.tools.mcp import MCPClient

# Import Strands telemetry for OpenTelemetry integration
from strands.telemetry import StrandsTelemetry

## Setting Strands Agents to emit LangFuse traces
#### The first step here is to set Strands Agents to emit traces to LangFuse

#### Configure the telemetry(Creates new tracer provider and sets it as global)

In [5]:
secret=get_secret()
secrets=json.loads(secret)

In [8]:
# Get keys for your project from the project settings page: https://cloud.langfuse.com
langfuse_public_key = secrets["LANGFUSE_PUBLIC_KEY"]
langfuse_secret_key = secrets["LANGFUSE_SECRET_KEY"]
langfuse_host = secrets['LANGFUSE_HOST'] #, 'https://cloud.langfuse.com')

if langfuse_public_key and langfuse_secret_key:
    LANGFUSE_AUTH = base64.b64encode(
        f"{langfuse_public_key}:{langfuse_secret_key}".encode()
    ).decode()
    
    os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = langfuse_host + "/api/public/otel"
    os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"
    print("Langfuse observability configured.\n")

else:
    print("Warning: Langfuse API keys not found. Observability will not be enabled.")

Langfuse observability configured.



In [9]:
try:
    strands_telemetry = StrandsTelemetry().setup_otlp_exporter()
    print("OpenTelemetry exporter configured for Langfuse.")
except ImportError:
    print("Warning: OpenTelemetry exporter not available. Install with 'pip install opentelemetry-exporter-otlp'.")
    strands_telemetry = None

OpenTelemetry exporter configured for Langfuse.


## Build your first strands agent
### Using the built in tool
Tools are the primary mechanism for extending agent capabilities, enabling them to perform actions beyond simple text generation. Tools allow agents to interact with external systems, access data, and manipulate their environment.
Strands offers built-in example tools to get started quickly experimenting with agents and tools during development. For more information, see Example [Built-in Tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/example-tools-package/).

In [11]:
# Customer query
query = """
What is in the picture of image/aws-architecture.png in the current folder path
"""

# Building the strands agent with built-in tool image_read and including Lanfuse for tracing
agent = Agent(
    tools=[image_reader],
    model="us.anthropic.claude-3-5-sonnet-20241022-v2:0", # Using Claude 3.5 Sonnet via Bedrock
    name="strands-agent-built-in-tool-example",
    trace_attributes={
        "session.id": "aws-mcp-agent-demo-session",
        "user.id": "xx-yy@example.com",
        "langfuse.tags": [
            "AWS-Strands-Agent",
            "Built-In-Tool",
        ],
        "metadata": {
            "environment": "development",
            "version": "1.0.0",
            "query_type": "storage_recommendation"
        }
    }
)
    
try:
    # Run the agent and get the response within the MCP client context
    response = agent(query)
except Exception as e:
    print(f"\nError: {e}")

Let me help you read that image using the image_reader function.
Tool #1: image_reader
The image shows an AWS (Amazon Web Services) architecture diagram for a microservice. It's divided into three main sections:

1. User Interface:
- Contains Amazon CloudFront (content delivery network)
- Connected to Amazon S3 (storage service)

2. Compute Implementation:
- Features an ALB (Application Load Balancer)
- Connected to Amazon ECS (Elastic Container Service)

3. Data Store:
- Contains three database services:
  * Amazon ElastiCache
  * Amazon Aurora
  * Amazon DynamoDB

The components are connected with arrows showing the flow of data/requests through the system. The entire architecture represents a microservice deployment on AWS, with front-end delivery through CloudFront, compute handling through containers on ECS, and data persistence across various database services.

### Now go to your Lanfuse console, you should be able to see a new trace with the name "strands_agent_built-in-tool-example" appear. Have a look at what's in the trace.

<div style="text-align:left">
    <img src="./image/trace1.jpg" width="15%" />
    <img src="./image/trace2.png" width="75%" />
</div>

## Strands Agents with custom tools
Define tools by creating Python modules that contain a tool specification and a matching function. This approach gives you more control over the tool's definition and is useful for dependency-free implementations of tools.

In [12]:
from typing import Dict, List, Optional
import requests
import xml.etree.ElementTree as ET

# Using the @tool python decorator, creating a custom tool that uses the AWS Health Dashboard RSS feed URL to check AWS service status
@tool
def aws_health_status_checker(region: str = "us-east-1", service_name: Optional[str] = None) -> Dict:
    """
    Check the current operational status of AWS services in a specific region using the public RSS feed.

    Args:
        region: AWS region to check (e.g., us-east-1, us-west-2)
        service_name: Optional specific service to check (e.g., ec2, s3, lambda)
                     If not provided, will return status for all services

    Returns:
        Dictionary containing service health information
    """
    try:
        # AWS Health Dashboard RSS feed URL
        rss_url = "https://status.aws.amazon.com/rss/all.rss"

        # Get the RSS feed
        response = requests.get(rss_url, timeout=10)
        response.raise_for_status()

        # Parse the XML
        root = ET.fromstring(response.content)

        # Find all items (service events)
        items = root.findall(".//item")

        # Filter events by region and service
        events = []
        for item in items:
            title = item.find("title").text if item.find("title") is not None else ""
            description = item.find("description").text if item.find("description") is not None else ""
            pub_date = item.find("pubDate").text if item.find("pubDate") is not None else ""
            link = item.find("link").text if item.find("link") is not None else ""

            # Check if this event is for the requested region
            if region.lower() in title.lower():
                # If service_name is specified, check if this event is for that service
                if service_name is None or service_name.lower() in title.lower():
                    events.append({
                        "title": title,
                        "description": description,
                        "published_date": pub_date,
                        "link": link
                    })

        if not events:
            return {
                "status": "healthy",
                "message": f"All services in {region} appear to be operating normally" if not service_name else f"{service_name} in {region} appears to be operating normally",
                "events": []
            }

        return {
            "status": "service_disruption",
            "message": f"Service disruptions detected in {region}",
            "events": events
        }

    except Exception as e:
        return {
            "status": "error",
            "message": f"Error checking AWS service status: {str(e)}",
            "events": []
        }

#### Create a Strands Agent with this custom tool

In [15]:
# Customer query
query = """
Check AWS service status in the us-east-1 region. focus on ec2 instances 
"""

# Building the strands agent with the custom tool "aws_health_status_checker" defined above 
agent = Agent(
    system_prompt="You are an AWS service checking agent, use the aws_service_status_checker to tell the user if the service status that they ask",
    tools=[aws_health_status_checker],
    model="us.anthropic.claude-3-5-sonnet-20241022-v2:0", # Using Claude 3.5 Sonnet via Bedrock
    name="strands-agent-custom-tool-example",
    trace_attributes={
        "session.id": "aws-mcp-agent-demo-session",
        "user.id": "example-user@example.com",
        "langfuse.tags": [
            "AWS-Strands-Agent",
            "Custom-Tool",
        ],
        "metadata": {
            "environment": "development",
            "version": "1.0.0",
            "query_type": "storage_recommendation"
        }
    }
)
    
try:
    # Run the agent and get the response within the MCP client context
    response = agent(query)
except Exception as e:
    print(f"\nError: {e}")

I'll help you check the status of EC2 service in the us-east-1 region using the aws_health_status_checker.
Tool #1: aws_health_status_checker
Based on the results, EC2 service in the us-east-1 region is currently healthy and operating normally. There are no reported issues or service disruptions at this time. You should be able to use EC2 instances in this region without any service-related problems.

Is there anything specific about EC2 or any other AWS service you'd like me to check for you?

### Now go to your Lanfuse console again, you should be able to see a new trace with the name "strands-agent-custom-tool-example" appear. Have a look at what's in the trace.

## Strands Agents with MCP servers
The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands Agents integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. Strands includes built-in support for connecting to MCP servers and using their tools.

When working with MCP tools in Strands, all agent operations must be performed within the MCP client's context manager (using a with statement). This requirement ensures that the MCP session remains active and connected while the agent is using the tools. If you attempt to use an agent or its MCP tools outside of this context, you'll encounter errors because the MCP session will have closed.

#### Initialize MCP client for AWS documentation and AWS Pricing

In [19]:
aws_docs_mcp = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", 
            args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Initialize MCP client for AWS pricing
aws_pricing_mcp = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", 
            args=["awslabs.aws-pricing-mcp-server@latest"]
        )
    )
)

#### Test query that requires both AWS documentation and cost estimation

In [22]:
print("AWS Pricing MCP Configuration:")
print(f"Command: {aws_pricing_mcp._command if hasattr(aws_pricing_mcp, '_command') else 'Unknown'}")
print(f"Args: {aws_pricing_mcp._args if hasattr(aws_pricing_mcp, '_args') else 'Unknown'}")
print(f"Env: {aws_pricing_mcp._env if hasattr(aws_pricing_mcp, '_env') else 'Unknown'}")


DEBUG:strands.tools.mcp.mcp_client:[Thread: MainThread, Session: de6b9c0a-9334-4ebb-84f6-8eb4e0743dc9] entering MCPClient context
DEBUG:strands.tools.mcp.mcp_client:[Thread: Thread-15 (_background_task), Session: de6b9c0a-9334-4ebb-84f6-8eb4e0743dc9] setting up background task event loop
DEBUG:strands.tools.mcp.mcp_client:[Thread: MainThread, Session: de6b9c0a-9334-4ebb-84f6-8eb4e0743dc9] background thread started, waiting for ready event
ERROR:strands.tools.mcp.mcp_client:client failed to initialize
  + Exception Group Traceback (most recent call last):
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 118, in start
  |     self._init_future.result(timeout=self._startup_timeout)
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 451, in result
  |     return self.__get_result()
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
  |     raise self._exception
  |   File "/opt/conda/

Attempting to start aws_docs_mcp...
aws_docs_mcp failed: the client initialization failed
Attempting to start aws_pricing_mcp...


DEBUG:strands.tools.mcp.mcp_client:[Thread: Thread-15 (_background_task), Session: de6b9c0a-9334-4ebb-84f6-8eb4e0743dc9] encountered exception on background thread after initialization unhandled errors in a TaskGroup (1 sub-exception)
ERROR:strands.tools.mcp.mcp_client:client failed to initialize
  + Exception Group Traceback (most recent call last):
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 118, in start
  |     self._init_future.result(timeout=self._startup_timeout)
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 458, in result
  |     return self.__get_result()
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
  |     raise self._exception
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 347, in _async_background_thread
  |     async with self._transport_callable() as (read_stream, write_stream, *_):
  |   File "/opt/conda/li

aws_pricing_mcp failed: the client initialization failed


In [21]:
query = """
I'm planning to use S3 for storing 1TB of data. 
1. What storage class would you recommend for data that is accessed infrequently based on the aws document?
2. Can you estimate the monthly cost for storing 1TB in this storage class?
"""

print(f"\n=== QUERY: {query} ===\n")
print("Processing query...")

# IMPORTANT: Use the MCP client within a context manager
with aws_docs_mcp, aws_pricing_mcp:
    # Get the available tools from the MCP server
    mcp_tools = aws_docs_mcp.list_tools_sync() + aws_pricing_mcp.list_tools_sync()
    print(f"Available MCP tools: {len(mcp_tools)} tools found")
    
    # Create the agent with our custom tools and MCP tools
    agent = Agent(
        system_prompt="""You are an AWS assistant specialized in helping users with AWS-related tasks.
        You can provide information about AWS services, estimate costs, and help manage AWS resources.
        
        When answering questions:
        1. Use the AWS documentation when needed to provide accurate information
        2. Provide clear explanations with examples
        3. Consider best practices and security recommendations
        4. Be precise and accurate in your responses
        
        For resource tagging, use the tag_aws_resources tool with the resource ID and tag key-value pairs.
        For AWS documentation queries, use the AWS documentation MCP tools.
        For cost estimation, use the AWS Pricing MCP tools.
        """,
        tools=[
            # MCP tools
            *mcp_tools
        ],
        model="us.anthropic.claude-3-5-sonnet-20241022-v2:0", # Using Claude 3.5 Sonnet via Bedrock
        name="strands-agent-mcp_example",
        trace_attributes={
            "session.id": "aws-mcp-agent-demo-session",
            "user.id": "example-user@example.com",
            "langfuse.tags": [
                "AWS-MCP-Agent",
                "MCP"
            ],
            "metadata": {
                "environment": "development",
                "version": "1.0.0",
                "query_type": "storage_recommendation"
            }
        }
    )
    
    try:
        # Run the agent and get the response within the MCP client context
        response = agent(query)
        #print("\nAgent messages:")
        #print(json.dumps(agent.messages, indent=4))
    except Exception as e:
        print(f"\nError: {e}")

client failed to initialize
  + Exception Group Traceback (most recent call last):
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 118, in start
  |     self._init_future.result(timeout=self._startup_timeout)
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 451, in result
  |     return self.__get_result()
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
  |     raise self._exception
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 118, in start
  |     self._init_future.result(timeout=self._startup_timeout)
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 458, in result
  |     return self.__get_result()
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
  |     raise self._exception
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py",


=== QUERY: 
I'm planning to use S3 for storing 1TB of data. 
1. What storage class would you recommend for data that is accessed infrequently based on the aws document?
2. Can you estimate the monthly cost for storing 1TB in this storage class?
 ===

Processing query...


  + Exception Group Traceback (most recent call last):
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 118, in start
  |     self._init_future.result(timeout=self._startup_timeout)
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 451, in result
  |     return self.__get_result()
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
  |     raise self._exception
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 118, in start
  |     self._init_future.result(timeout=self._startup_timeout)
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 458, in result
  |     return self.__get_result()
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
  |     raise self._exception
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 347, in _async_backgro

#### Now you know the drill, go to your Lanfuse console, you should be able to see a new trace with the name "strands-agent-mc-example" appear. Have a look at what's in the trace.

#### You can also use the built in tracing to show what the agent traces locally

In [ ]:
print("\nAgent messages:")
print(json.dumps(agent.messages, indent=4))

You can go to the Langfuse console to see the details of the traces.

## Lab4 Summary:

In Lab 4, we explored advanced observability for **Strands Agents** using `Langfuse`, demonstrating three distinct
tool integration approaches with comprehensive tracing capabilities. 

We successfully implemented built-in tools using the `image_reader` for AWS architecture analysis, created custom tools like the 
`aws_health_status_checker` for real-time service monitoring, and integrated `MCP (Model Context Protocol)` tools combining AWS Documentation and Pricing servers for complex multi-tool workflows. The key achievement was establishing complete observability through OpenTelemetry integration with StrandsTelemetry, enabling real-time monitoring of agent decision-making processes, tool usage patterns, and performance metrics in the Langfuse console. 

As we conclude this lab, you've gained valuable experience in building transparent, monitorable AI agents that transform from black boxes into production-ready systems with comprehensive observability, preparing you for deploying robust agentic systems with confidence in enterprise environments.

# Evaluating Strands Agent with Observability with LangFuse

## Overview
In this module, we'll explore how to implement observability for [Strands Agent](https://strandsagents.com/latest/) using [Langfuse](https://langfuse.com/). Strands Agent enables you to build an intelligent AWS assistant by connecting foundation models to AWS services and resources, here's the link to [Strands Agent SDK's GitHub repo](https://github.com/strands-agents). While powerful AI agents can solve complex problems, understanding their behavior and performance requires robust observability solutions.

Langfuse integration provides comprehensive visibility into your agent's operations, helping you monitor, debug, and optimize its performance. This module will guide you through implementing and leveraging Langfuse observability with Strands Agent to create more reliable, transparent, and effective AI solutions.

Here's what you will learn from this module:
1. Setup Lanfuse Observability 
2. Create a Strands Agent with [built-in tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/example-tools-package/) and trace it with Lanfuse
3. Create a Strands Agent with [custom tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/python-tools/) and trace it with Lanfuse
4. Lastly create a Strands Agent with [MCP tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/mcp-tools/) and trace it with Lanfuse

## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                              |
|--------------------|---------------------------------------------------------|
|Native tools used   |image_reader                                             |
|Custom tools created|aws_health_status_checker                                |
|MCP Tools           |aws_documentation_mcp_tool, aws_pricing_mcp_tool         |
|Agent Structure     |Single agent architecture                                |
|AWS services used   |Amazon Bedrock                                           |
|Integrations        |LangFuse for observability                               |

</div>



## Architecture

<div style="text-align:left">
    <img src="./image/architecture.png" width="75%" />
</div>

## Key Features
- Fetches Strands agent interaction traces from Langfuse. You can also save these traces offline and use them here without Langfuse.
- Evaluates conversations using specialized metrics for agents, tools
- Pushes evaluation scores back to Langfuse for a complete feedback loop
- Evaluate both single-turn (with context) and multi-turn conversations

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.5 on Amazon Bedrock
* LangFuse Key

Let's now install the requirement packages for our Strands Agent

In [2]:
# Uncomment the following line to install dependencies if you are not using AWS workshop environment
# %pip install -q langfuse boto3  --upgrade

In [ ]:
# Install required packages
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 9.9 MB/s eta 0:00:00
  Using cached opentelemetry_exporter_otlp-1.36.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached docstring_parser-0.17.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached opentelemetry_api-1.36.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_instrumentation_threading-0.57b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_sdk-1.36.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached watchdog-6.0.0-py3-none-manylinux2014_x86_64.whl.metadata (44 kB)
  Using cached aws_requests_auth-0.4.3-py2.py3-none-any.whl.metadata (567 bytes)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached pillow-11.3.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (9.0 kB)
  Using cached prompt_toolkit-3.0.51-py3-none-any.whl.metadata (6.4 kB)
  Using cached PyJWT-2.10.1-py3-n

In [3]:
# if you already define the environment variables in the .env of the vscode server, please skip the following cell
# Define the environment variables for langfuse
# You can find those values when you create the API key in Langfuse
# import os
# os.environ["LANGFUSE_SECRET_KEY"] = "xxxx" # Your Langfuse project secret key
# os.environ["LANGFUSE_PUBLIC_KEY"] = "xxxx" # Your Langfuse project public key
# os.environ["LANGFUSE_HOST"] = "xxx" # Langfuse domain

### Importing dependency packages

Now let's import the dependency packages

In [4]:
import os
import json
import base64
from typing import Dict, Any, List, Optional
from datetime import datetime
import uv

from strands import Agent, tool
from strands_tools import calculator, image_reader
import sys
sys.path.append(os.path.abspath('..'))  # Add parent directory to path

from utils import *

# Import MCP related modules
from mcp import StdioServerParameters, stdio_client
from strands.tools.mcp import MCPClient

# Import Strands telemetry for OpenTelemetry integration
from strands.telemetry import StrandsTelemetry

## Setting Strands Agents to emit LangFuse traces
#### The first step here is to set Strands Agents to emit traces to LangFuse

#### Configure the telemetry(Creates new tracer provider and sets it as global)

In [5]:
secret=get_secret()
secrets=json.loads(secret)

In [8]:
# Get keys for your project from the project settings page: https://cloud.langfuse.com
langfuse_public_key = secrets["LANGFUSE_PUBLIC_KEY"]
langfuse_secret_key = secrets["LANGFUSE_SECRET_KEY"]
langfuse_host = secrets['LANGFUSE_HOST'] #, 'https://cloud.langfuse.com')

if langfuse_public_key and langfuse_secret_key:
    LANGFUSE_AUTH = base64.b64encode(
        f"{langfuse_public_key}:{langfuse_secret_key}".encode()
    ).decode()
    
    os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = langfuse_host + "/api/public/otel"
    os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"
    print("Langfuse observability configured.\n")

else:
    print("Warning: Langfuse API keys not found. Observability will not be enabled.")

Langfuse observability configured.



In [9]:
try:
    strands_telemetry = StrandsTelemetry().setup_otlp_exporter()
    print("OpenTelemetry exporter configured for Langfuse.")
except ImportError:
    print("Warning: OpenTelemetry exporter not available. Install with 'pip install opentelemetry-exporter-otlp'.")
    strands_telemetry = None

OpenTelemetry exporter configured for Langfuse.


## Build your first strands agent
### Using the built in tool
Tools are the primary mechanism for extending agent capabilities, enabling them to perform actions beyond simple text generation. Tools allow agents to interact with external systems, access data, and manipulate their environment.
Strands offers built-in example tools to get started quickly experimenting with agents and tools during development. For more information, see Example [Built-in Tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/example-tools-package/).

In [11]:
# Customer query
query = """
What is in the picture of image/aws-architecture.png in the current folder path
"""

# Building the strands agent with built-in tool image_read and including Lanfuse for tracing
agent = Agent(
    tools=[image_reader],
    model="us.anthropic.claude-3-5-sonnet-20241022-v2:0", # Using Claude 3.5 Sonnet via Bedrock
    name="strands-agent-built-in-tool-example",
    trace_attributes={
        "session.id": "aws-mcp-agent-demo-session",
        "user.id": "xx-yy@example.com",
        "langfuse.tags": [
            "AWS-Strands-Agent",
            "Built-In-Tool",
        ],
        "metadata": {
            "environment": "development",
            "version": "1.0.0",
            "query_type": "storage_recommendation"
        }
    }
)
    
try:
    # Run the agent and get the response within the MCP client context
    response = agent(query)
except Exception as e:
    print(f"\nError: {e}")

Let me help you read that image using the image_reader function.
Tool #1: image_reader
The image shows an AWS (Amazon Web Services) architecture diagram for a microservice. It's divided into three main sections:

1. User Interface:
- Contains Amazon CloudFront (content delivery network)
- Connected to Amazon S3 (storage service)

2. Compute Implementation:
- Features an ALB (Application Load Balancer)
- Connected to Amazon ECS (Elastic Container Service)

3. Data Store:
- Contains three database services:
  * Amazon ElastiCache
  * Amazon Aurora
  * Amazon DynamoDB

The components are connected with arrows showing the flow of data/requests through the system. The entire architecture represents a microservice deployment on AWS, with front-end delivery through CloudFront, compute handling through containers on ECS, and data persistence across various database services.

### Now go to your Lanfuse console, you should be able to see a new trace with the name "strands_agent_built-in-tool-example" appear. Have a look at what's in the trace.

<div style="text-align:left">
    <img src="./image/trace1.jpg" width="15%" />
    <img src="./image/trace2.png" width="75%" />
</div>

## Strands Agents with custom tools
Define tools by creating Python modules that contain a tool specification and a matching function. This approach gives you more control over the tool's definition and is useful for dependency-free implementations of tools.

In [12]:
from typing import Dict, List, Optional
import requests
import xml.etree.ElementTree as ET

# Using the @tool python decorator, creating a custom tool that uses the AWS Health Dashboard RSS feed URL to check AWS service status
@tool
def aws_health_status_checker(region: str = "us-east-1", service_name: Optional[str] = None) -> Dict:
    """
    Check the current operational status of AWS services in a specific region using the public RSS feed.

    Args:
        region: AWS region to check (e.g., us-east-1, us-west-2)
        service_name: Optional specific service to check (e.g., ec2, s3, lambda)
                     If not provided, will return status for all services

    Returns:
        Dictionary containing service health information
    """
    try:
        # AWS Health Dashboard RSS feed URL
        rss_url = "https://status.aws.amazon.com/rss/all.rss"

        # Get the RSS feed
        response = requests.get(rss_url, timeout=10)
        response.raise_for_status()

        # Parse the XML
        root = ET.fromstring(response.content)

        # Find all items (service events)
        items = root.findall(".//item")

        # Filter events by region and service
        events = []
        for item in items:
            title = item.find("title").text if item.find("title") is not None else ""
            description = item.find("description").text if item.find("description") is not None else ""
            pub_date = item.find("pubDate").text if item.find("pubDate") is not None else ""
            link = item.find("link").text if item.find("link") is not None else ""

            # Check if this event is for the requested region
            if region.lower() in title.lower():
                # If service_name is specified, check if this event is for that service
                if service_name is None or service_name.lower() in title.lower():
                    events.append({
                        "title": title,
                        "description": description,
                        "published_date": pub_date,
                        "link": link
                    })

        if not events:
            return {
                "status": "healthy",
                "message": f"All services in {region} appear to be operating normally" if not service_name else f"{service_name} in {region} appears to be operating normally",
                "events": []
            }

        return {
            "status": "service_disruption",
            "message": f"Service disruptions detected in {region}",
            "events": events
        }

    except Exception as e:
        return {
            "status": "error",
            "message": f"Error checking AWS service status: {str(e)}",
            "events": []
        }

#### Create a Strands Agent with this custom tool

In [15]:
# Customer query
query = """
Check AWS service status in the us-east-1 region. focus on ec2 instances 
"""

# Building the strands agent with the custom tool "aws_health_status_checker" defined above 
agent = Agent(
    system_prompt="You are an AWS service checking agent, use the aws_service_status_checker to tell the user if the service status that they ask",
    tools=[aws_health_status_checker],
    model="us.anthropic.claude-3-5-sonnet-20241022-v2:0", # Using Claude 3.5 Sonnet via Bedrock
    name="strands-agent-custom-tool-example",
    trace_attributes={
        "session.id": "aws-mcp-agent-demo-session",
        "user.id": "example-user@example.com",
        "langfuse.tags": [
            "AWS-Strands-Agent",
            "Custom-Tool",
        ],
        "metadata": {
            "environment": "development",
            "version": "1.0.0",
            "query_type": "storage_recommendation"
        }
    }
)
    
try:
    # Run the agent and get the response within the MCP client context
    response = agent(query)
except Exception as e:
    print(f"\nError: {e}")

I'll help you check the status of EC2 service in the us-east-1 region using the aws_health_status_checker.
Tool #1: aws_health_status_checker
Based on the results, EC2 service in the us-east-1 region is currently healthy and operating normally. There are no reported issues or service disruptions at this time. You should be able to use EC2 instances in this region without any service-related problems.

Is there anything specific about EC2 or any other AWS service you'd like me to check for you?

### Now go to your Lanfuse console again, you should be able to see a new trace with the name "strands-agent-custom-tool-example" appear. Have a look at what's in the trace.

## Strands Agents with MCP servers
The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands Agents integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. Strands includes built-in support for connecting to MCP servers and using their tools.

When working with MCP tools in Strands, all agent operations must be performed within the MCP client's context manager (using a with statement). This requirement ensures that the MCP session remains active and connected while the agent is using the tools. If you attempt to use an agent or its MCP tools outside of this context, you'll encounter errors because the MCP session will have closed.

#### Initialize MCP client for AWS documentation and AWS Pricing

In [25]:
import subprocess
import shutil

# Check if uvx is available
uvx_path = shutil.which("uvx")
print(f"uvx command found at: {uvx_path}")

if not uvx_path:
    print("❌ uvx command not found")
    # Try alternative commands
    for alt_cmd in ["uv", "pip", "pipx"]:
        if shutil.which(alt_cmd):
            print(f"✅ Alternative found: {alt_cmd}")

uvx command found at: /opt/conda/bin/uvx


In [27]:
from strands.tools.mcp import MCPClient
from mcp.client.stdio import StdioServerParameters, stdio_client
import sys

# Use direct Python module execution
aws_docs_mcp = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command=sys.executable,
            args=["-m", "aws_documentation_mcp_server"]  # Adjust module name
        )
    )
)
aws_pricing_mcp = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command=sys.executable,
            args=["-m", "aws_pricing_mcp_server"]  # Adjust module name
        )
    )
)

DEBUG:strands.tools.mcp.mcp_client:[Thread: MainThread, Session: f8bd8ed9-ce0d-4ae1-85a0-5d3392a4caae] initializing MCPClient connection
DEBUG:strands.tools.mcp.mcp_client:[Thread: MainThread, Session: f9ac3af1-2e83-48bc-bf4a-fba93ebef317] initializing MCPClient connection


aws_docs_mcp = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", 
            args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Initialize MCP client for AWS pricing
aws_pricing_mcp = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", 
            args=["awslabs.aws-pricing-mcp-server@latest"]
        )
    )
)

print("AWS Pricing MCP Configuration:")
print(f"Command: {aws_pricing_mcp._command if hasattr(aws_pricing_mcp, '_command') else 'Unknown'}")
print(f"Args: {aws_pricing_mcp._args if hasattr(aws_pricing_mcp, '_args') else 'Unknown'}")
print(f"Env: {aws_pricing_mcp._env if hasattr(aws_pricing_mcp, '_env') else 'Unknown'}")


#### Test query that requires both AWS documentation and cost estimation

In [31]:
mcp_tools = []

# Initialize AWS Docs MCP
try:
    with aws_docs_mcp:
        docs_tools = aws_docs_mcp.list_tools_sync()
        mcp_tools.extend(docs_tools)
        print(f"AWS Docs MCP: {len(docs_tools)} tools loaded")
except Exception as e:
    print(f"AWS Docs MCP failed: {e}")

DEBUG:strands.tools.mcp.mcp_client:[Thread: MainThread, Session: f8bd8ed9-ce0d-4ae1-85a0-5d3392a4caae] entering MCPClient context
DEBUG:strands.tools.mcp.mcp_client:[Thread: Thread-18 (_background_task), Session: f8bd8ed9-ce0d-4ae1-85a0-5d3392a4caae] setting up background task event loop
DEBUG:strands.tools.mcp.mcp_client:[Thread: MainThread, Session: f8bd8ed9-ce0d-4ae1-85a0-5d3392a4caae] background thread started, waiting for ready event
ERROR:strands.tools.mcp.mcp_client:client failed to initialize
  + Exception Group Traceback (most recent call last):
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 118, in start
  |     self._init_future.result(timeout=self._startup_timeout)
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 451, in result
  |     return self.__get_result()
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
  |     raise self._exception
  |   File "/opt/conda/

AWS Docs MCP failed: the client initialization failed


DEBUG:asyncio:Using selector: EpollSelector
DEBUG:strands.tools.mcp.mcp_client:[Thread: Thread-18 (_background_task), Session: f8bd8ed9-ce0d-4ae1-85a0-5d3392a4caae] starting async background thread for MCP connection
DEBUG:strands.tools.mcp.mcp_client:[Thread: Thread-18 (_background_task), Session: f8bd8ed9-ce0d-4ae1-85a0-5d3392a4caae] transport connection established
DEBUG:strands.tools.mcp.mcp_client:[Thread: Thread-18 (_background_task), Session: f8bd8ed9-ce0d-4ae1-85a0-5d3392a4caae] initializing MCP session
DEBUG:strands.tools.mcp.mcp_client:[Thread: Thread-18 (_background_task), Session: f8bd8ed9-ce0d-4ae1-85a0-5d3392a4caae] encountered exception on background thread after initialization unhandled errors in a TaskGroup (1 sub-exception)


In [29]:
query = """
I'm planning to use S3 for storing 1TB of data. 
1. What storage class would you recommend for data that is accessed infrequently based on the aws document?
2. Can you estimate the monthly cost for storing 1TB in this storage class?
"""

print(f"\n=== QUERY: {query} ===\n")
print("Processing query...")

# IMPORTANT: Use the MCP client within a context manager
with aws_docs_mcp, aws_pricing_mcp:
    # Get the available tools from the MCP server
    mcp_tools = aws_docs_mcp.list_tools_sync() + aws_pricing_mcp.list_tools_sync()
    print(f"Available MCP tools: {len(mcp_tools)} tools found")
    
    # Create the agent with our custom tools and MCP tools
    agent = Agent(
        system_prompt="""You are an AWS assistant specialized in helping users with AWS-related tasks.
        You can provide information about AWS services, estimate costs, and help manage AWS resources.
        
        When answering questions:
        1. Use the AWS documentation when needed to provide accurate information
        2. Provide clear explanations with examples
        3. Consider best practices and security recommendations
        4. Be precise and accurate in your responses
        
        For resource tagging, use the tag_aws_resources tool with the resource ID and tag key-value pairs.
        For AWS documentation queries, use the AWS documentation MCP tools.
        For cost estimation, use the AWS Pricing MCP tools.
        """,
        tools=[
            # MCP tools
            *mcp_tools
        ],
        model="us.anthropic.claude-3-5-sonnet-20241022-v2:0", # Using Claude 3.5 Sonnet via Bedrock
        name="strands-agent-mcp_example",
        trace_attributes={
            "session.id": "aws-mcp-agent-demo-session",
            "user.id": "example-user@example.com",
            "langfuse.tags": [
                "AWS-MCP-Agent",
                "MCP"
            ],
            "metadata": {
                "environment": "development",
                "version": "1.0.0",
                "query_type": "storage_recommendation"
            }
        }
    )
    
    try:
        # Run the agent and get the response within the MCP client context
        response = agent(query)
        #print("\nAgent messages:")
        #print(json.dumps(agent.messages, indent=4))
    except Exception as e:
        print(f"\nError: {e}")

DEBUG:strands.tools.mcp.mcp_client:[Thread: MainThread, Session: f8bd8ed9-ce0d-4ae1-85a0-5d3392a4caae] entering MCPClient context
DEBUG:strands.tools.mcp.mcp_client:[Thread: MainThread, Session: f8bd8ed9-ce0d-4ae1-85a0-5d3392a4caae] background thread started, waiting for ready event



=== QUERY: 
I'm planning to use S3 for storing 1TB of data. 
1. What storage class would you recommend for data that is accessed infrequently based on the aws document?
2. Can you estimate the monthly cost for storing 1TB in this storage class?
 ===

Processing query...


DEBUG:strands.tools.mcp.mcp_client:[Thread: Thread-17 (_background_task), Session: f8bd8ed9-ce0d-4ae1-85a0-5d3392a4caae] transport connection established
DEBUG:strands.tools.mcp.mcp_client:[Thread: Thread-17 (_background_task), Session: f8bd8ed9-ce0d-4ae1-85a0-5d3392a4caae] initializing MCP session


MCPClientInitializationError: background thread did not start in 30 seconds

#### Now you know the drill, go to your Lanfuse console, you should be able to see a new trace with the name "strands-agent-mc-example" appear. Have a look at what's in the trace.

#### You can also use the built in tracing to show what the agent traces locally

In [ ]:
print("\nAgent messages:")
print(json.dumps(agent.messages, indent=4))

You can go to the Langfuse console to see the details of the traces.

## Lab4 Summary:

In Lab 4, we explored advanced observability for **Strands Agents** using `Langfuse`, demonstrating three distinct
tool integration approaches with comprehensive tracing capabilities. 

We successfully implemented built-in tools using the `image_reader` for AWS architecture analysis, created custom tools like the 
`aws_health_status_checker` for real-time service monitoring, and integrated `MCP (Model Context Protocol)` tools combining AWS Documentation and Pricing servers for complex multi-tool workflows. The key achievement was establishing complete observability through OpenTelemetry integration with StrandsTelemetry, enabling real-time monitoring of agent decision-making processes, tool usage patterns, and performance metrics in the Langfuse console. 

As we conclude this lab, you've gained valuable experience in building transparent, monitorable AI agents that transform from black boxes into production-ready systems with comprehensive observability, preparing you for deploying robust agentic systems with confidence in enterprise environments.

# Evaluating Strands Agent with Observability with LangFuse

## Overview
In this module, we'll explore how to implement observability for [Strands Agent](https://strandsagents.com/latest/) using [Langfuse](https://langfuse.com/). Strands Agent enables you to build an intelligent AWS assistant by connecting foundation models to AWS services and resources, here's the link to [Strands Agent SDK's GitHub repo](https://github.com/strands-agents). While powerful AI agents can solve complex problems, understanding their behavior and performance requires robust observability solutions.

Langfuse integration provides comprehensive visibility into your agent's operations, helping you monitor, debug, and optimize its performance. This module will guide you through implementing and leveraging Langfuse observability with Strands Agent to create more reliable, transparent, and effective AI solutions.

Here's what you will learn from this module:
1. Setup Lanfuse Observability 
2. Create a Strands Agent with [built-in tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/example-tools-package/) and trace it with Lanfuse
3. Create a Strands Agent with [custom tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/python-tools/) and trace it with Lanfuse
4. Lastly create a Strands Agent with [MCP tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/mcp-tools/) and trace it with Lanfuse

## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                              |
|--------------------|---------------------------------------------------------|
|Native tools used   |image_reader                                             |
|Custom tools created|aws_health_status_checker                                |
|MCP Tools           |aws_documentation_mcp_tool, aws_pricing_mcp_tool         |
|Agent Structure     |Single agent architecture                                |
|AWS services used   |Amazon Bedrock                                           |
|Integrations        |LangFuse for observability                               |

</div>



## Architecture

<div style="text-align:left">
    <img src="./image/architecture.png" width="75%" />
</div>

## Key Features
- Fetches Strands agent interaction traces from Langfuse. You can also save these traces offline and use them here without Langfuse.
- Evaluates conversations using specialized metrics for agents, tools
- Pushes evaluation scores back to Langfuse for a complete feedback loop
- Evaluate both single-turn (with context) and multi-turn conversations

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.5 on Amazon Bedrock
* LangFuse Key

Let's now install the requirement packages for our Strands Agent

In [2]:
# Uncomment the following line to install dependencies if you are not using AWS workshop environment
# %pip install -q langfuse boto3  --upgrade

In [ ]:
# Install required packages
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 9.9 MB/s eta 0:00:00
  Using cached opentelemetry_exporter_otlp-1.36.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached docstring_parser-0.17.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached opentelemetry_api-1.36.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_instrumentation_threading-0.57b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_sdk-1.36.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached watchdog-6.0.0-py3-none-manylinux2014_x86_64.whl.metadata (44 kB)
  Using cached aws_requests_auth-0.4.3-py2.py3-none-any.whl.metadata (567 bytes)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached pillow-11.3.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (9.0 kB)
  Using cached prompt_toolkit-3.0.51-py3-none-any.whl.metadata (6.4 kB)
  Using cached PyJWT-2.10.1-py3-n

In [3]:
# if you already define the environment variables in the .env of the vscode server, please skip the following cell
# Define the environment variables for langfuse
# You can find those values when you create the API key in Langfuse
# import os
# os.environ["LANGFUSE_SECRET_KEY"] = "xxxx" # Your Langfuse project secret key
# os.environ["LANGFUSE_PUBLIC_KEY"] = "xxxx" # Your Langfuse project public key
# os.environ["LANGFUSE_HOST"] = "xxx" # Langfuse domain

### Importing dependency packages

Now let's import the dependency packages

In [4]:
import os
import json
import base64
from typing import Dict, Any, List, Optional
from datetime import datetime
import uv

from strands import Agent, tool
from strands_tools import calculator, image_reader
import sys
sys.path.append(os.path.abspath('..'))  # Add parent directory to path

from utils import *

# Import MCP related modules
from mcp import StdioServerParameters, stdio_client
from strands.tools.mcp import MCPClient

# Import Strands telemetry for OpenTelemetry integration
from strands.telemetry import StrandsTelemetry

## Setting Strands Agents to emit LangFuse traces
#### The first step here is to set Strands Agents to emit traces to LangFuse

#### Configure the telemetry(Creates new tracer provider and sets it as global)

In [5]:
secret=get_secret()
secrets=json.loads(secret)

In [8]:
# Get keys for your project from the project settings page: https://cloud.langfuse.com
langfuse_public_key = secrets["LANGFUSE_PUBLIC_KEY"]
langfuse_secret_key = secrets["LANGFUSE_SECRET_KEY"]
langfuse_host = secrets['LANGFUSE_HOST'] #, 'https://cloud.langfuse.com')

if langfuse_public_key and langfuse_secret_key:
    LANGFUSE_AUTH = base64.b64encode(
        f"{langfuse_public_key}:{langfuse_secret_key}".encode()
    ).decode()
    
    os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = langfuse_host + "/api/public/otel"
    os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"
    print("Langfuse observability configured.\n")

else:
    print("Warning: Langfuse API keys not found. Observability will not be enabled.")

Langfuse observability configured.



In [9]:
try:
    strands_telemetry = StrandsTelemetry().setup_otlp_exporter()
    print("OpenTelemetry exporter configured for Langfuse.")
except ImportError:
    print("Warning: OpenTelemetry exporter not available. Install with 'pip install opentelemetry-exporter-otlp'.")
    strands_telemetry = None

OpenTelemetry exporter configured for Langfuse.


## Build your first strands agent
### Using the built in tool
Tools are the primary mechanism for extending agent capabilities, enabling them to perform actions beyond simple text generation. Tools allow agents to interact with external systems, access data, and manipulate their environment.
Strands offers built-in example tools to get started quickly experimenting with agents and tools during development. For more information, see Example [Built-in Tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/example-tools-package/).

In [11]:
# Customer query
query = """
What is in the picture of image/aws-architecture.png in the current folder path
"""

# Building the strands agent with built-in tool image_read and including Lanfuse for tracing
agent = Agent(
    tools=[image_reader],
    model="us.anthropic.claude-3-5-sonnet-20241022-v2:0", # Using Claude 3.5 Sonnet via Bedrock
    name="strands-agent-built-in-tool-example",
    trace_attributes={
        "session.id": "aws-mcp-agent-demo-session",
        "user.id": "xx-yy@example.com",
        "langfuse.tags": [
            "AWS-Strands-Agent",
            "Built-In-Tool",
        ],
        "metadata": {
            "environment": "development",
            "version": "1.0.0",
            "query_type": "storage_recommendation"
        }
    }
)
    
try:
    # Run the agent and get the response within the MCP client context
    response = agent(query)
except Exception as e:
    print(f"\nError: {e}")

Let me help you read that image using the image_reader function.
Tool #1: image_reader
The image shows an AWS (Amazon Web Services) architecture diagram for a microservice. It's divided into three main sections:

1. User Interface:
- Contains Amazon CloudFront (content delivery network)
- Connected to Amazon S3 (storage service)

2. Compute Implementation:
- Features an ALB (Application Load Balancer)
- Connected to Amazon ECS (Elastic Container Service)

3. Data Store:
- Contains three database services:
  * Amazon ElastiCache
  * Amazon Aurora
  * Amazon DynamoDB

The components are connected with arrows showing the flow of data/requests through the system. The entire architecture represents a microservice deployment on AWS, with front-end delivery through CloudFront, compute handling through containers on ECS, and data persistence across various database services.

### Now go to your Lanfuse console, you should be able to see a new trace with the name "strands_agent_built-in-tool-example" appear. Have a look at what's in the trace.

<div style="text-align:left">
    <img src="./image/trace1.jpg" width="15%" />
    <img src="./image/trace2.png" width="75%" />
</div>

## Strands Agents with custom tools
Define tools by creating Python modules that contain a tool specification and a matching function. This approach gives you more control over the tool's definition and is useful for dependency-free implementations of tools.

In [12]:
from typing import Dict, List, Optional
import requests
import xml.etree.ElementTree as ET

# Using the @tool python decorator, creating a custom tool that uses the AWS Health Dashboard RSS feed URL to check AWS service status
@tool
def aws_health_status_checker(region: str = "us-east-1", service_name: Optional[str] = None) -> Dict:
    """
    Check the current operational status of AWS services in a specific region using the public RSS feed.

    Args:
        region: AWS region to check (e.g., us-east-1, us-west-2)
        service_name: Optional specific service to check (e.g., ec2, s3, lambda)
                     If not provided, will return status for all services

    Returns:
        Dictionary containing service health information
    """
    try:
        # AWS Health Dashboard RSS feed URL
        rss_url = "https://status.aws.amazon.com/rss/all.rss"

        # Get the RSS feed
        response = requests.get(rss_url, timeout=10)
        response.raise_for_status()

        # Parse the XML
        root = ET.fromstring(response.content)

        # Find all items (service events)
        items = root.findall(".//item")

        # Filter events by region and service
        events = []
        for item in items:
            title = item.find("title").text if item.find("title") is not None else ""
            description = item.find("description").text if item.find("description") is not None else ""
            pub_date = item.find("pubDate").text if item.find("pubDate") is not None else ""
            link = item.find("link").text if item.find("link") is not None else ""

            # Check if this event is for the requested region
            if region.lower() in title.lower():
                # If service_name is specified, check if this event is for that service
                if service_name is None or service_name.lower() in title.lower():
                    events.append({
                        "title": title,
                        "description": description,
                        "published_date": pub_date,
                        "link": link
                    })

        if not events:
            return {
                "status": "healthy",
                "message": f"All services in {region} appear to be operating normally" if not service_name else f"{service_name} in {region} appears to be operating normally",
                "events": []
            }

        return {
            "status": "service_disruption",
            "message": f"Service disruptions detected in {region}",
            "events": events
        }

    except Exception as e:
        return {
            "status": "error",
            "message": f"Error checking AWS service status: {str(e)}",
            "events": []
        }

#### Create a Strands Agent with this custom tool

In [15]:
# Customer query
query = """
Check AWS service status in the us-east-1 region. focus on ec2 instances 
"""

# Building the strands agent with the custom tool "aws_health_status_checker" defined above 
agent = Agent(
    system_prompt="You are an AWS service checking agent, use the aws_service_status_checker to tell the user if the service status that they ask",
    tools=[aws_health_status_checker],
    model="us.anthropic.claude-3-5-sonnet-20241022-v2:0", # Using Claude 3.5 Sonnet via Bedrock
    name="strands-agent-custom-tool-example",
    trace_attributes={
        "session.id": "aws-mcp-agent-demo-session",
        "user.id": "example-user@example.com",
        "langfuse.tags": [
            "AWS-Strands-Agent",
            "Custom-Tool",
        ],
        "metadata": {
            "environment": "development",
            "version": "1.0.0",
            "query_type": "storage_recommendation"
        }
    }
)
    
try:
    # Run the agent and get the response within the MCP client context
    response = agent(query)
except Exception as e:
    print(f"\nError: {e}")

I'll help you check the status of EC2 service in the us-east-1 region using the aws_health_status_checker.
Tool #1: aws_health_status_checker
Based on the results, EC2 service in the us-east-1 region is currently healthy and operating normally. There are no reported issues or service disruptions at this time. You should be able to use EC2 instances in this region without any service-related problems.

Is there anything specific about EC2 or any other AWS service you'd like me to check for you?

### Now go to your Lanfuse console again, you should be able to see a new trace with the name "strands-agent-custom-tool-example" appear. Have a look at what's in the trace.

## Strands Agents with MCP servers
The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands Agents integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. Strands includes built-in support for connecting to MCP servers and using their tools.

When working with MCP tools in Strands, all agent operations must be performed within the MCP client's context manager (using a with statement). This requirement ensures that the MCP session remains active and connected while the agent is using the tools. If you attempt to use an agent or its MCP tools outside of this context, you'll encounter errors because the MCP session will have closed.

#### Initialize MCP client for AWS documentation and AWS Pricing

In [19]:
aws_docs_mcp = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", 
            args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Initialize MCP client for AWS pricing
aws_pricing_mcp = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", 
            args=["awslabs.aws-pricing-mcp-server@latest"]
        )
    )
)

#### Test query that requires both AWS documentation and cost estimation

In [22]:
print("AWS Pricing MCP Configuration:")
print(f"Command: {aws_pricing_mcp._command if hasattr(aws_pricing_mcp, '_command') else 'Unknown'}")
print(f"Args: {aws_pricing_mcp._args if hasattr(aws_pricing_mcp, '_args') else 'Unknown'}")
print(f"Env: {aws_pricing_mcp._env if hasattr(aws_pricing_mcp, '_env') else 'Unknown'}")


DEBUG:strands.tools.mcp.mcp_client:[Thread: MainThread, Session: de6b9c0a-9334-4ebb-84f6-8eb4e0743dc9] entering MCPClient context
DEBUG:strands.tools.mcp.mcp_client:[Thread: Thread-15 (_background_task), Session: de6b9c0a-9334-4ebb-84f6-8eb4e0743dc9] setting up background task event loop
DEBUG:strands.tools.mcp.mcp_client:[Thread: MainThread, Session: de6b9c0a-9334-4ebb-84f6-8eb4e0743dc9] background thread started, waiting for ready event
ERROR:strands.tools.mcp.mcp_client:client failed to initialize
  + Exception Group Traceback (most recent call last):
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 118, in start
  |     self._init_future.result(timeout=self._startup_timeout)
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 451, in result
  |     return self.__get_result()
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
  |     raise self._exception
  |   File "/opt/conda/

Attempting to start aws_docs_mcp...
aws_docs_mcp failed: the client initialization failed
Attempting to start aws_pricing_mcp...


DEBUG:strands.tools.mcp.mcp_client:[Thread: Thread-15 (_background_task), Session: de6b9c0a-9334-4ebb-84f6-8eb4e0743dc9] encountered exception on background thread after initialization unhandled errors in a TaskGroup (1 sub-exception)
ERROR:strands.tools.mcp.mcp_client:client failed to initialize
  + Exception Group Traceback (most recent call last):
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 118, in start
  |     self._init_future.result(timeout=self._startup_timeout)
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 458, in result
  |     return self.__get_result()
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
  |     raise self._exception
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 347, in _async_background_thread
  |     async with self._transport_callable() as (read_stream, write_stream, *_):
  |   File "/opt/conda/li

aws_pricing_mcp failed: the client initialization failed


In [21]:
query = """
I'm planning to use S3 for storing 1TB of data. 
1. What storage class would you recommend for data that is accessed infrequently based on the aws document?
2. Can you estimate the monthly cost for storing 1TB in this storage class?
"""

print(f"\n=== QUERY: {query} ===\n")
print("Processing query...")

# IMPORTANT: Use the MCP client within a context manager
with aws_docs_mcp, aws_pricing_mcp:
    # Get the available tools from the MCP server
    mcp_tools = aws_docs_mcp.list_tools_sync() + aws_pricing_mcp.list_tools_sync()
    print(f"Available MCP tools: {len(mcp_tools)} tools found")
    
    # Create the agent with our custom tools and MCP tools
    agent = Agent(
        system_prompt="""You are an AWS assistant specialized in helping users with AWS-related tasks.
        You can provide information about AWS services, estimate costs, and help manage AWS resources.
        
        When answering questions:
        1. Use the AWS documentation when needed to provide accurate information
        2. Provide clear explanations with examples
        3. Consider best practices and security recommendations
        4. Be precise and accurate in your responses
        
        For resource tagging, use the tag_aws_resources tool with the resource ID and tag key-value pairs.
        For AWS documentation queries, use the AWS documentation MCP tools.
        For cost estimation, use the AWS Pricing MCP tools.
        """,
        tools=[
            # MCP tools
            *mcp_tools
        ],
        model="us.anthropic.claude-3-5-sonnet-20241022-v2:0", # Using Claude 3.5 Sonnet via Bedrock
        name="strands-agent-mcp_example",
        trace_attributes={
            "session.id": "aws-mcp-agent-demo-session",
            "user.id": "example-user@example.com",
            "langfuse.tags": [
                "AWS-MCP-Agent",
                "MCP"
            ],
            "metadata": {
                "environment": "development",
                "version": "1.0.0",
                "query_type": "storage_recommendation"
            }
        }
    )
    
    try:
        # Run the agent and get the response within the MCP client context
        response = agent(query)
        #print("\nAgent messages:")
        #print(json.dumps(agent.messages, indent=4))
    except Exception as e:
        print(f"\nError: {e}")

client failed to initialize
  + Exception Group Traceback (most recent call last):
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 118, in start
  |     self._init_future.result(timeout=self._startup_timeout)
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 451, in result
  |     return self.__get_result()
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
  |     raise self._exception
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 118, in start
  |     self._init_future.result(timeout=self._startup_timeout)
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 458, in result
  |     return self.__get_result()
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
  |     raise self._exception
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py",


=== QUERY: 
I'm planning to use S3 for storing 1TB of data. 
1. What storage class would you recommend for data that is accessed infrequently based on the aws document?
2. Can you estimate the monthly cost for storing 1TB in this storage class?
 ===

Processing query...


  + Exception Group Traceback (most recent call last):
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 118, in start
  |     self._init_future.result(timeout=self._startup_timeout)
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 451, in result
  |     return self.__get_result()
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
  |     raise self._exception
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 118, in start
  |     self._init_future.result(timeout=self._startup_timeout)
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 458, in result
  |     return self.__get_result()
  |   File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
  |     raise self._exception
  |   File "/opt/conda/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 347, in _async_backgro

#### Now you know the drill, go to your Lanfuse console, you should be able to see a new trace with the name "strands-agent-mc-example" appear. Have a look at what's in the trace.

#### You can also use the built in tracing to show what the agent traces locally

In [ ]:
print("\nAgent messages:")
print(json.dumps(agent.messages, indent=4))

You can go to the Langfuse console to see the details of the traces.

## Lab4 Summary:

In Lab 4, we explored advanced observability for **Strands Agents** using `Langfuse`, demonstrating three distinct
tool integration approaches with comprehensive tracing capabilities. 

We successfully implemented built-in tools using the `image_reader` for AWS architecture analysis, created custom tools like the 
`aws_health_status_checker` for real-time service monitoring, and integrated `MCP (Model Context Protocol)` tools combining AWS Documentation and Pricing servers for complex multi-tool workflows. The key achievement was establishing complete observability through OpenTelemetry integration with StrandsTelemetry, enabling real-time monitoring of agent decision-making processes, tool usage patterns, and performance metrics in the Langfuse console. 

As we conclude this lab, you've gained valuable experience in building transparent, monitorable AI agents that transform from black boxes into production-ready systems with comprehensive observability, preparing you for deploying robust agentic systems with confidence in enterprise environments.